# **Analisis estadistico**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df = pd.read_csv('/content/process_online_retail_data.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df['invoicedate'] = pd.to_datetime(df['invoicedate'])
df.loc[:, 'date'] = df['invoicedate'].dt.date

In [4]:
df.head()

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,total_pay,hour,day,weekday,month,year,date
0,536365,85123a,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,united kingdom,15.30,8,1,2,12,2010,2010-12-01
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,united kingdom,20.34,8,1,2,12,2010,2010-12-01
2,536365,84406b,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,united kingdom,22.00,8,1,2,12,2010,2010-12-01
3,536365,84029g,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,united kingdom,20.34,8,1,2,12,2010,2010-12-01
4,536365,84029e,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,united kingdom,20.34,8,1,2,12,2010,2010-12-01


In [5]:
df[['quantity', 'unitprice','total_pay']].describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,535187.0,11.302279,218.980977,1.00,1.00,3.00,11.00,80995.0
unitprice,535187.0,4.645242,97.364810,-11062.06,1.25,2.08,4.13,38970.0
total_pay,535187.0,21.513913,380.997864,-11062.06,3.90,9.90,17.70,168469.6


In [6]:
# Revisamos metricas sin incluir el valor atipico negativo
df_mins_unitprice = df[df['unitprice'] > 0]
df_mins_unitprice[['quantity', 'unitprice','total_pay']].describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,534129.0,10.948629,216.402375,1.000,1.00,3.00,11.00,80995.0
unitprice,534129.0,4.695864,95.079189,0.001,1.25,2.10,4.13,38970.0
total_pay,534129.0,21.597948,380.770267,0.001,3.90,9.92,17.70,168469.6


* El valor negativo del error en el debito no afecta la distribución de los datos, pero es un factor importante a tener en cuenta para el negocio.

* La cantidad de artículos tiene una desviación estándar relativamente alta de 216.40, lo que sugiere una gran variabilidad en la cantidad de artículos comprados en cada transacción. <br>

Hay una **variabilidad significativa** en las cantidades de artículos comprados y en los precios unitarios por cada compra/facturación. También podemos observar que el total de pago puede tener valores extremos debido a la alta desviación estándar.

## Comportamiento transaccional en los paises

In [7]:
countries_list = df['country'].unique()
countries_list

array(['united kingdom', 'france', 'australia', 'netherlands', 'germany',
       'norway', 'eire', 'switzerland', 'spain', 'poland', 'portugal',
       'italy', 'belgium', 'lithuania', 'japan', 'iceland',
       'channel islands', 'denmark', 'cyprus', 'sweden', 'austria',
       'israel', 'finland', 'bahrain', 'greece', 'hong kong', 'singapore',
       'lebanon', 'united arab emirates', 'saudi arabia',
       'czech republic', 'canada', 'unspecified', 'brazil', 'usa',
       'european community', 'malta', 'rsa'], dtype=object)

In [8]:
coordinates = {
    'united kingdom': (51.509865, -0.118092),
    'france': (48.856613, 2.352222),
    'australia': (-25.274398, 133.775136),
    'netherlands': (52.370216, 4.895168),
    'germany': (51.165691, 10.451526),
    'norway': (60.472024, 8.468946),
    'eire': (53.349805, -6.26031),
    'switzerland': (46.818188, 8.227512),
    'spain': (40.463667, -3.74922),
    'poland': (51.919438, 19.145136),
    'portugal': (39.399872, -8.224454),
    'italy': (41.87194, 12.56738),
    'belgium': (50.850346, 4.351721),
    'lithuania': (55.169438, 23.881275),
    'japan': (36.204824, 138.252924),
    'iceland': (64.963051, -19.020835),
    'channel islands': (49.214439, -2.13125),
    'denmark': (56.26392, 9.501785),
    'cyprus': (35.126413, 33.429859),
    'sweden': (60.128161, 18.643501),
    'austria': (47.516231, 14.550072),
    'israel': (31.046051, 34.851612),
    'finland': (61.92411, 25.748151),
    'bahrain': (26.0667, 50.5577),
    'greece': (39.074208, 21.824312),
    'hong kong': (22.396428, 114.109497),
    'singapore': (1.352083, 103.819839),
    'lebanon': (33.854721, 35.862285),
    'united arab emirates': (23.424076, 53.847818),
    'saudi arabia': (23.885942, 45.079162),
    'czech republic': (49.817492, 15.472962),
    'canada': (56.130366, -106.346771),
    'brazil': (-14.235004, -51.92528),
    'usa': (37.09024, -95.712891),
    'european community': (44.547659, 18.676128),  # se calculo usando el punto central de los paises de la union europea
    'malta': (35.937496, 14.375416),
    'rsa': (-30.559482, 22.937506)
}


In [9]:
inv_country = df['country'].value_counts()
transaction_for_country = pd.DataFrame(inv_country).reset_index()
transaction_for_country = transaction_for_country.rename(columns = {'country':'invoices', 'index':'country'})

total_invoices = transaction_for_country['invoices'].sum()
transaction_for_country['% invoices'] = ((transaction_for_country['invoices'] / total_invoices) * 100).round(3)

# Calcular el total de dinero facturado por país y fusionar en el DataFrame 'transaction_for_country'
total_money_by_country = df.groupby('country')['total_pay'].sum().reset_index()
transaction_for_country = pd.merge(transaction_for_country, total_money_by_country, on='country')

# Calcular el total de items facturados por país y fusionar en el DataFrame 'transaction_for_country'
total_items_by_country = df.groupby('country')['quantity'].sum().reset_index()
transaction_for_country = pd.merge(transaction_for_country, total_items_by_country, on='country')

# Calcular el total de clientes unicos por país y fusionar en el DataFrame 'transaction_for_country'
customers_id = df[df['customerid'] > 0]
total_users_by_country = customers_id.groupby('country')['customerid'].nunique().reset_index()
transaction_for_country = pd.merge(transaction_for_country, total_users_by_country, on='country')
transaction_for_country = transaction_for_country.rename(columns = {'customerid':'customers'})

# Agregar coordenadas a los paises
transaction_for_country['latitude'] = transaction_for_country['country'].map(lambda x: coordinates.get(x, (0, 0))[0])
transaction_for_country['longitude'] = transaction_for_country['country'].map(lambda x: coordinates.get(x, (0, 0))[1])

transaction_for_country.head()

,country,invoices,% invoices,total_pay,quantity,customers,latitude,longitude
0,united kingdom,488846,91.341,9792111.764,5107627,3950,51.509865,-0.118092
1,germany,9480,1.771,235847.330,120971,95,51.165691,10.451526
2,france,8541,1.596,221933.630,113684,87,48.856613,2.352222
3,eire,8184,1.529,303287.660,152067,3,53.349805,-6.260310
4,spain,2528,0.472,68361.090,29071,31,40.463667,-3.749220


In [10]:
not_greater_invoice_countries =  transaction_for_country[transaction_for_country['country'] != 'united kingdom']
not_greater_invoice_countries_resume = pd.DataFrame({
                                                      'country' : 'others_countries',
                                                      'invoices': [not_greater_invoice_countries['invoices'].sum()],
                                                      '% invoices': [not_greater_invoice_countries['% invoices'].sum()],
                                                      'total_pay': [not_greater_invoice_countries['total_pay'].sum()],
                                                      'quantity': [not_greater_invoice_countries['quantity'].sum()],
                                                      'customers': [not_greater_invoice_countries['customers'].sum()]
                                                    })

greater_country = transaction_for_country[transaction_for_country['country'] == 'united kingdom']

greater_vs_nongreater = pd.concat([greater_country, not_greater_invoice_countries_resume])
greater_vs_nongreater = greater_vs_nongreater.drop(columns = ['latitude','longitude'])
greater_vs_nongreater

,country,invoices,% invoices,total_pay,quantity,customers
0,united kingdom,488846,91.341,9792111.764,5107627,3950
0,others_countries,46057,8.607,1700796.890,936489,430


In [11]:
# Crear el gráfico de cloropletas con Plotly
fig = go.Figure(data=go.Choropleth(
    locations=transaction_for_country['country'],  # Nombres de los países
    z=transaction_for_country['invoices'],  # Datos numéricos para el color del país
    locationmode='country names',  # Modo de ubicación usando nombres de países
    colorscale='Bluered',  # Escala de colores
    colorbar_title='# compras',  # Título de la barra de colores
))

fig.update_layout(
    title_text='Volumen de compras por pais',  # Título del gráfico
    geo=dict(
        showframe=True,  # No mostrar marcos de países
        showcoastlines=True,  # No mostrar costas de países
        projection_type='natural earth'  # Tipo de proyección del mapa: natural earth
    )
)

fig.show()

In [12]:
fig = px.pie(greater_vs_nongreater, values='invoices', names='country', title="Volumen de compras por país")
fig.show()

El 91% de los registros son de compras en el Reino Unido, por lo tanto, el core de negocio esta en este pais.

Este analisis se propone desde dos aristas: Reino Unido y los demas paises. De esta manera podemos diagnosticar la diferencia de comportamiento transaccional en las diferentes verticales del negocio: que pueden ser geografia consolidada y expansion

In [13]:
user_in_countries = df.groupby(['customerid'])['country'].nunique().reset_index()
user_in_countries = user_in_countries.rename(columns = {'country':'count_country'}).sort_values(by='count_country', ascending = False)
uic = user_in_countries.count_country.value_counts()
users_buy_countries = pd.DataFrame(uic).reset_index()
users_buy_countries = users_buy_countries.rename( columns = {'index':'nums_country','count_country':'nums_customer'})

total_users = users_buy_countries['nums_customer'].sum()
users_buy_countries['%_users'] = (users_buy_countries['nums_customer'] / total_users)*100
users_buy_countries = users_buy_countries[users_buy_countries['nums_country'] < 5]
users_buy_countries

,nums_country,nums_customer,%_users
0,1,4364,99.794192
1,2,8,0.182941


In [14]:
fig = px.pie(users_buy_countries, values='%_users', names='nums_country')
fig.show()

Solo 8 usuarios han realizado compras en dos paises diferentes. El 99,8% solo compra en un pais. Aunque el dato no es representativo, es importante para el negocio tener presente esta situación, hay que revisar el volumen transaccional de estos usuarios

### **Reino Unido**

In [15]:
df_united_kingdom= df[(df['country'] == 'united kingdom') & (df['unitprice'] > 0)]
df_united_kingdom

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,total_pay,hour,day,weekday,month,year,date
0,536365,85123a,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,united kingdom,15.30,8,1,2,12,2010,2010-12-01
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,united kingdom,20.34,8,1,2,12,2010,2010-12-01
2,536365,84406b,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,united kingdom,22.00,8,1,2,12,2010,2010-12-01
3,536365,84029g,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,united kingdom,20.34,8,1,2,12,2010,2010-12-01
4,536365,84029e,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,united kingdom,20.34,8,1,2,12,2010,2010-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535167,581585,22466,fairy tale cottage night light,12,2011-12-09 12:31:00,1.95,15804,united kingdom,23.40,12,9,4,12,2011,2011-12-09
535168,581586,22061,large cake stand hanging strawbery,8,2011-12-09 12:49:00,2.95,13113,united kingdom,23.60,12,9,4,12,2011,2011-12-09
535169,581586,23275,set of 3 hanging owls ollie beak,24,2011-12-09 12:49:00,1.25,13113,united kingdom,30.00,12,9,4,12,2011,2011-12-09
535170,581586,21217,red retrospot round cake tins,24,2011-12-09 12:49:00,8.95,13113,united kingdom,214.80,12,9,4,12,2011,2011-12-09


In [16]:
df_united_kingdom[['quantity', 'unitprice','total_pay']].describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,487806.0,10.061274,225.961068,1.000,1.00,3.0,10.00,80995.0
unitprice,487806.0,4.623501,97.551762,0.001,1.25,2.1,4.13,38970.0
total_pay,487806.0,20.119137,397.133353,0.001,3.44,8.5,17.00,168469.6


Para los registros filtrados de Reino Unido, la varianza de los datos sigue siendo alta. Es posible que los registros que no tienen customerid esten afectando los datos, vamos a revisar

In [17]:
df_uk_noid_customer = df_united_kingdom[df_united_kingdom['customerid'] > 0]
df_uk_noid_customer[['quantity', 'unitprice','total_pay']].describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,356704.0,12.616825,264.109945,1.000,2.00,4.00,12.00,80995.0
unitprice,356704.0,3.268475,71.164719,0.001,1.25,1.95,3.75,38970.0
total_pay,356704.0,21.931050,455.036979,0.001,4.20,10.20,17.85,168469.6


La varianza de los datos sigue siendo alta. Entonces podriamos encontrar patrones en el consumo de los clientes, es decir, que unos compran mas que otros.

### Flujo de inventario y moneda

In [18]:
month_time_invoices = df_united_kingdom.groupby(['year', 'month'])['invoiceno'].count().reset_index()
month_time_pays = df_united_kingdom.groupby(['year', 'month'])['total_pay'].sum().reset_index().round(2)
total_items_uk = df_united_kingdom.groupby(['year', 'month'])['quantity'].sum().reset_index()

# Fusionar 'month_time_invoices' y 'month_time_pays' en el DataFrame 'month_time_transactions'
month_time_transactions = pd.merge(month_time_invoices, month_time_pays, on=['year', 'month'])

# Fusionar 'month_time_transactions' con 'total_items_uk'
month_time_transactions = pd.merge(month_time_transactions, total_items_uk, on=['year', 'month'])

# Crear la columna 'month_year'
month_time_transactions['month_year'] = month_time_transactions['year'].astype(str) + '-' + month_time_transactions['month'].astype(str)


month_time_transactions

,year,month,invoiceno,total_pay,quantity,month_year
0,2010,12,39359,817548.58,328010,2010-12
1,2011,1,31518,686956.26,394476,2011-1
2,2011,2,25015,450716.57,231611,2011-2
3,2011,3,32930,611162.67,308737,2011-3
4,2011,4,27563,509940.50,287956,2011-4
5,2011,5,33184,680959.39,326824,2011-5
6,2011,6,32728,683360.85,310780,2011-6
7,2011,7,35537,637432.12,337988,2011-7
8,2011,8,30794,653574.84,340938,2011-8
9,2011,9,45013,930092.92,484777,2011-9


In [19]:
# Crear el gráfico con Plotly
fig = go.Figure()

# Gráfico de barras para el total_pay de transacciones por mes
fig.add_trace(go.Bar(x=month_time_transactions['month_year'], y=month_time_transactions['total_pay'], marker=dict(color='blue'), name='Total dinero'))

# Gráfico de línea para la cantidad de items vendidos por mes
fig.add_trace(go.Scatter(x=month_time_transactions['month_year'], y=month_time_transactions['quantity'], mode='lines+markers', marker=dict(size=8), line=dict(color='red'), name='# Items', yaxis='y2'))

# Gráfico de barras para el número de invoices por mes en el eje y derecho
fig.add_trace(go.Scatter(x=month_time_transactions['month_year'], y=month_time_transactions['invoiceno'], mode='lines+markers', marker=dict(size=8), line=dict(color='black'), name='# invoices', yaxis='y2'))

# Configurar el diseño del gráfico
fig.update_layout(title_text='Flujo de inventario e ingresos por mes', xaxis_title='Meses', yaxis=dict(title='Total ingresos'), yaxis2=dict(title='# Items/Invoices', overlaying='y', side='right'), hovermode='closest')

fig.show()

In [20]:
day_time_invoices = df_united_kingdom.groupby(['date'])['invoiceno'].count().reset_index()
day_time_pays = df_united_kingdom.groupby(['date'])['total_pay'].sum().reset_index().round(2)
total_items_uk = df_united_kingdom.groupby(['date'])['quantity'].sum().reset_index()

# Fusionar 'day_time_invoices' y 'day_time_pays' en el DataFrame 'day_time_transactions'
day_time_transactions = pd.merge(day_time_invoices, day_time_pays, on=['date'])

# Fusionar 'day_time_transactions' con 'total_items_uk'
day_time_transactions = pd.merge(day_time_transactions, total_items_uk, on=['date'])

day_time_transactions

,date,invoiceno,total_pay,quantity
0,2010-12-01,2895,54837.01,24056
1,2010-12-02,2051,48968.17,41393
2,2010-12-03,1961,42586.86,12058
3,2010-12-05,2388,25833.88,13508
4,2010-12-06,3758,54035.09,21156
...,...,...,...,...
300,2011-12-05,5153,116069.61,42825
301,2011-12-06,3092,54780.05,29481
302,2011-12-07,1936,55524.65,29852
303,2011-12-08,4733,79684.57,33319


In [21]:
# Crear el gráfico con Plotly
fig = go.Figure()

# Gráfico de barras para el total_pay de transacciones por mes
fig.add_trace(go.Bar(x=day_time_transactions['date'], y=day_time_transactions['total_pay'], marker=dict(color='blue'), name='Total ventas'))

# Gráfico de línea para la cantidad de items vendidos por mes
fig.add_trace(go.Scatter(x=day_time_transactions['date'], y=day_time_transactions['quantity'], mode='lines+markers', marker=dict(size=2), line=dict(color='red'), name='# Items', yaxis='y2'))

# Gráfico de barras para el número de invoices por mes en el eje y derecho
fig.add_trace(go.Scatter(x=day_time_transactions['date'], y=day_time_transactions['invoiceno'], mode='lines+markers', marker=dict(size=2), line=dict(color='black'), name='# invoices', yaxis='y2'))

# Configurar el diseño del gráfico
fig.update_layout(title_text='Flujo de inventario e ingresos por dia', xaxis_title='Dias', yaxis=dict(title='Total ingresos'), yaxis2=dict(title='# Items/Invoices', overlaying='y', side='right'), hovermode='closest')

fig.show()

In [22]:
weekday_time_invoices = df_united_kingdom.groupby(['weekday'])['invoiceno'].count().reset_index()
weekday_time_pays = df_united_kingdom.groupby(['weekday'])['total_pay'].sum().reset_index().round(2)
total_items_uk = df_united_kingdom.groupby(['weekday'])['quantity'].sum().reset_index()

# Fusionar 'day_time_invoices' y 'day_time_pays' en el DataFrame 'day_time_transactions'
weekday_time_transactions = pd.merge(weekday_time_invoices, weekday_time_pays, on=['weekday'])

# Fusionar 'day_time_transactions' con 'total_items_uk'
weekday_time_transactions = pd.merge(weekday_time_transactions, total_items_uk, on=['weekday'])

weekday_time_transactions

,weekday,invoiceno,total_pay,quantity
0,0,86388,1690115.83,756569
1,1,92658,2082570.06,1038768
2,2,84785,1640613.68,862824
3,3,92039,1854633.72,1003211
4,4,72369,1830910.72,833458
5,6,59567,715391.87,413120


In [23]:
# Crear el gráfico con Plotly
fig = go.Figure()

# Gráfico de barras para el total_pay de transacciones por mes
fig.add_trace(go.Bar(x=weekday_time_transactions['weekday'], y=weekday_time_transactions['total_pay'], marker=dict(color='blue'), name='Total ventas'))

# Gráfico de línea para la cantidad de items vendidos por mes
fig.add_trace(go.Scatter(x=weekday_time_transactions['weekday'], y=weekday_time_transactions['quantity'], mode='lines+markers', marker=dict(size=8), line=dict(color='red'), name='# Items', yaxis='y2'))

# Gráfico de barras para el número de invoices por mes en el eje y derecho
fig.add_trace(go.Scatter(x=weekday_time_transactions['weekday'], y=weekday_time_transactions['invoiceno'], mode='lines+markers', marker=dict(size=8), line=dict(color='black'), name='# invoices', yaxis='y2'))

# Configurar el diseño del gráfico
fig.update_layout(title_text='Flujo de inventario e ingresos por dia de la semana', xaxis_title='dia de la semana', yaxis=dict(title='Total ingresos'), yaxis2=dict(title='# Items/Invoices', overlaying='y', side='right'), hovermode='closest')

fig.show()

In [24]:
# Trafico de usuarios por hora y numero de invoices
hour_count_invoices = df_united_kingdom.groupby(['hour'])['invoiceno'].count().reset_index()
user_count_buys = df_united_kingdom.groupby(['hour'])['customerid'].nunique().reset_index()

users_hour_invoices = pd.merge(hour_count_invoices, user_count_buys, on=['hour'])
users_hour_invoices = users_hour_invoices.rename(columns = {'invoiceno':'num_invoices','customerid':'num_customers'})
users_hour_invoices = users_hour_invoices.sort_values(by='hour', ascending = True)
users_hour_invoices

,hour,num_invoices,num_customers
0,6,27,18
1,7,218,24
2,8,6536,395
3,9,29233,858
4,10,41562,1232
5,11,51770,1321
6,12,70893,1587
7,13,64666,1524
8,14,59905,1372
9,15,72272,1285


In [25]:
# Crear el gráfico con Plotly
fig = go.Figure()
fig.add_trace(go.Bar(x=users_hour_invoices['hour'], y=users_hour_invoices['num_invoices'], marker=dict(color='blue'), name='total compras'))
fig.add_trace(go.Scatter(x=users_hour_invoices['hour'], y=users_hour_invoices['num_customers'], mode='lines+markers', marker=dict(size=6), line=dict(color='red'), name='# customers', yaxis='y2'))

fig.update_layout(title_text='Actividad de usuarios por hora', xaxis_title='hora', yaxis=dict(title='total compras'), yaxis2=dict(title='# customers', overlaying='y', side='right'), hovermode='closest')
fig.show()

### Descriptivo comportamiento de los usuarios

In [44]:
invoices_by_user = df_uk_noid_customer.groupby(['customerid'])['invoiceno'].count().reset_index()

total_items_by_user = df_uk_noid_customer.groupby(['customerid'])['quantity'].sum().reset_index()
max_item_user_by_buy = df_uk_noid_customer.groupby(['customerid'])['quantity'].max().reset_index()
min_item_user_by_buy = df_uk_noid_customer.groupby(['customerid'])['quantity'].min().reset_index()
avg_item_user_by_buy = df_uk_noid_customer.groupby(['customerid'])['quantity'].mean().reset_index()

total_pay_by_user = df_uk_noid_customer.groupby(['customerid'])['total_pay'].sum().reset_index()
avg_pay_user_by_buy = df_uk_noid_customer.groupby(['customerid'])['total_pay'].mean().reset_index()

grouped_counts = df_uk_noid_customer.groupby(['customerid', 'month']).size().reset_index(name='records_per_month')
months_per_customer = grouped_counts.groupby('customerid')['month'].nunique().reset_index(name='months_with_buys')

result = invoices_by_user.merge(total_items_by_user, on='customerid', suffixes=('_invoices', '_total_items'))
result = result.merge(max_item_user_by_buy, on='customerid', suffixes=('', '_max_item'))
result = result.merge(min_item_user_by_buy, on='customerid', suffixes=('', '_min_item'))
result = result.merge(avg_item_user_by_buy, on='customerid', suffixes=('', '_avg_item')).round().astype(int)
result = result.merge(total_pay_by_user, on='customerid', suffixes=('', '_total_pay'))
result = result.merge(avg_pay_user_by_buy, on='customerid', suffixes=('', '_avg_pay')).round(2)
result = result.merge(months_per_customer, on='customerid')

user_behavior_uk = result.rename(columns={
    'invoiceno': 'total_invoices',
    'quantity': 'total_items',
    'quantity_max_item': 'max_items_by_buy',
    'quantity_min_item': 'min_items_by_buy',
    'quantity_avg_item': 'avg_items_by_buy',
    'total_pay': 'total_pay',
    'total_pay_avg_pay': 'avg_pay_by_buy'
})

user_behavior_uk['avg_buys_per_month'] = (user_behavior_uk['total_invoices'] / user_behavior_uk['months_with_buys']).round().astype(int)
user_behavior_uk

,customerid,total_invoices,total_items,max_items_by_buy,min_items_by_buy,avg_items_by_buy,total_pay,avg_pay_by_buy,months_with_buys,avg_buys_per_month
0,12346,2,148430,74215,74215,74215,154367.20,77183.60,1,2
1,12747,103,1275,48,1,12,4196.01,40.74,8,13
2,12748,4458,26825,576,1,6,37700.82,8.46,12,372
3,12749,231,1520,75,1,7,4313.56,18.67,4,58
4,12820,59,722,48,4,12,942.34,15.97,4,15
...,...,...,...,...,...,...,...,...,...,...
3944,18280,10,45,8,2,4,180.60,18.06,1,10
3945,18281,7,54,12,1,8,80.82,11.55,1,7
3946,18282,13,108,48,1,8,179.50,13.81,2,6
3947,18283,721,1357,13,1,2,2045.53,2.84,10,72


In [46]:
def assign_group(total_invoices):
    if 1 <= total_invoices <= 5:
        return '01-05'
    elif 6 <= total_invoices <= 10:
        return '06-10'
    elif 11 <= total_invoices <= 15:
        return '11-15'
    else:
        return 'Más de 15'

In [48]:
count_buys_per_customer = user_behavior_uk.total_invoices.value_counts()
buys_per_customer = pd.DataFrame(count_buys_per_customer).reset_index()
buys_per_customer = buys_per_customer.rename(columns = {'index':'nums_customers'})
buys_per_customer['group'] = buys_per_customer['total_invoices'].apply(assign_group)


group_counts = buys_per_customer.groupby(['group'])['nums_customers'].sum().reset_index()
group_counts

,group,nums_customers
0,01-05,138262
1,06-10,6616
2,11-15,2709
3,Más de 15,2724


In [51]:
fig = px.bar(group_counts, y='nums_customers')
fig.show()

In [77]:
months_users = user_behavior_uk.months_with_buys.value_counts()
buy_month_users = pd.DataFrame(months_users).reset_index()
buy_month_users = buy_month_users.rename(columns = {'index':'meses_compra','months_with_buys':'count_customers'})

fig = px.bar(buy_month_users, x='meses_compra', y='count_customers', text_auto='.1s', title="Frecuencia de compra clientes", color_discrete_sequence=['blue'])
fig.update_traces(textfont_size=15, textangle=0, textposition="outside", cliponaxis=False)
fig.update_xaxes(title_text="# meses")
fig.update_yaxes(title_text="# customers")
fig.update_layout(height=800)
fig.show()


La mayoria de los usuarios en Reino Unido han hecho entre 1 a 5 transacciones. Tambien un comportamiento comun es que la mayoria de los usuarios han comprado una unica vez.

Una estrategia de clasificación de estos clientes puede ser en función de su frecuencia de compra y numero de compras. Hya una cantidad interesante de usuarios que han comprado > 5 meses.

In [82]:
# Crear el scatter plot
fig = go.Figure()

# Agregar los puntos al scatter plot
fig.add_trace(go.Scatter(
    y=user_behavior_uk.total_pay,
    x=user_behavior_uk.total_items,
    mode='markers',
    marker=dict(size=10, color='blue')
))

# Agregar título al gráfico
fig.update_layout(title='Relación entre el numero de items comprados y el pago')

# Agregar nombre a los ejes
fig.update_xaxes(title_text='# items')
fig.update_yaxes(title_text='total pago')
fig.show()

En este grafico podemos observar la relacion del total de facturación con el numero de items comprados. Hay una relacion directamente proporcional entre el numero de items comprados y el total pagado, sin embargo, en este grafico es interesante ver los puntos que no siguen esa funcion lineal. Pueden indicar usuarios que compran productos de mayor valor o que incluso tiene mayor capacidad adquisitiva

### **Componente internacional**

In [47]:
df_countries= df[df['country'] != 'united kingdom']
df_countries

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country,total_pay,hour,day,weekday,month,year
26,536370,22728,alarm clock bakelike pink,24,2010-12-01 08:45:00,3.75,12583,france,90.00,8,1,2,12,2010
27,536370,22727,alarm clock bakelike red,24,2010-12-01 08:45:00,3.75,12583,france,90.00,8,1,2,12,2010
28,536370,22726,alarm clock bakelike green,12,2010-12-01 08:45:00,3.75,12583,france,45.00,8,1,2,12,2010
29,536370,21724,panda and bunnies sticker sheet,12,2010-12-01 08:45:00,0.85,12583,france,10.20,8,1,2,12,2010
30,536370,21883,stars gift tape,24,2010-12-01 08:45:00,0.65,12583,france,15.60,8,1,2,12,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535182,581587,22613,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,france,10.20,12,9,4,12,2011
535183,581587,22899,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,france,12.60,12,9,4,12,2011
535184,581587,23254,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,france,16.60,12,9,4,12,2011
535185,581587,23255,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,france,16.60,12,9,4,12,2011


In [48]:
df_countries[['quantity', 'unitprice','total_pay']].describe().T

,count,mean,std,min,25%,50%,75%,max
quantity,46341.0,20.310438,47.006357,1.0,4.00,10.00,16.00,2400.00
unitprice,46341.0,5.455760,63.421841,0.0,1.25,1.95,3.75,4161.06
total_pay,46341.0,37.156182,103.320773,0.0,12.75,17.40,30.00,8322.12


In [218]:
not_greater_invoice_countries = not_greater_invoice_countries.sort_values(by='invoices', ascending=True)
not_greater_invoice_countries.head()

,country,invoices,% invoices,total_pay,quantity,customers,latitude,longitude
36,saudi arabia,10,0.002,160.67,85,1,23.885942,45.079162
35,bahrain,19,0.004,959.88,368,2,26.066700,50.557700
34,czech republic,30,0.006,945.76,750,1,49.817492,15.472962
33,brazil,32,0.006,1143.60,356,1,-14.235004,-51.925280
32,lithuania,35,0.007,1661.06,652,1,55.169438,23.881275


In [216]:
# Crear el gráfico de cloropletas con Plotly
fig = go.Figure(data=go.Choropleth(
    locations=not_greater_invoice_countries['country'],  # Nombres de los países
    z=transaction_for_country['invoices'],  # Datos numéricos para el color del país
    locationmode='country names',  # Modo de ubicación usando nombres de países
    colorscale='Bluered',  # Escala de colores
    colorbar_title='# compras',  # Título de la barra de colores
))

fig.update_layout(
    title_text='Volumen de compras por pais',  # Título del gráfico
    geo=dict(
        showframe=True,  # No mostrar marcos de países
        showcoastlines=True,  # No mostrar costas de países
        projection_type='natural earth'  # Tipo de proyección del mapa: natural earth
    )
)

fig.show()

### Flujo de inventario y moneda

In [242]:
fig = px.bar(not_greater_invoice_countries, x='invoices', y='country', text_auto='.1s', title="Volumen de compras por país", color_discrete_sequence=['blue'])
fig.update_traces(textfont_size=15, textangle=0, textposition="outside", cliponaxis=False)
fig.update_xaxes(title_text="# compras")
fig.update_yaxes(title_text="country")
fig.update_layout(height=800)
fig.update_xaxes(tickangle=270)
fig.show()

In [308]:
nums_transactions_month = df_countries.groupby(['year','month','country'])['invoiceno'].count().reset_index()
total_pay_month = df_countries.groupby(['year','month','country'])['total_pay'].sum().reset_index()
quantity_per_month = df_countries.groupby(['year','month','country'])['quantity'].sum().reset_index()
user_per_month = df_countries.groupby(['year','month','country'])['customerid'].nunique().reset_index()


merged_df = pd.merge(nums_transactions_month, total_pay_month, on=['year', 'month', 'country'])
merged_df = pd.merge(merged_df, quantity_per_month, on=['year', 'month', 'country'])
merged_df = pd.merge(merged_df, user_per_month, on=['year', 'month', 'country'])
merged_df = merged_df.rename(columns={'customerid': 'num_users'})
merged_df['month_year'] = merged_df['year'].astype(str) + '-' + merged_df['month'].astype(str)
merged_df


,year,month,country,invoiceno,total_pay,quantity,num_users,month_year
0,2010,12,australia,34,993.10,470,3,2010-12
1,2010,12,austria,9,297.36,99,2,2010-12
2,2010,12,bahrain,1,205.74,54,1,2010-12
3,2010,12,belgium,95,1809.91,1755,4,2010-12
4,2010,12,channel islands,17,363.53,80,1,2010-12
...,...,...,...,...,...,...,...,...
296,2011,12,norway,73,2785.70,2319,2,2011-12
297,2011,12,portugal,102,2814.43,1572,5,2011-12
298,2011,12,spain,71,360.99,292,3,2011-12
299,2011,12,sweden,2,476.00,80,1,2011-12


In [316]:
def plot_country_data(country):
    df_country = merged_df[merged_df['country'] == country]

    # Crear el gráfico con Plotly
    fig = go.Figure()

    # Gráfico de barras para el total_pay de transacciones por mes
    fig.add_trace(go.Bar(x=df_country['month_year'], y=df_country['total_pay'], marker=dict(color='blue'), name='Total dinero'))

    # Gráfico de línea para la cantidad de items vendidos por mes
    fig.add_trace(go.Scatter(x=df_country['month_year'], y=df_country['quantity'], mode='lines+markers', marker=dict(size=8), line=dict(color='red'), name='# Items', yaxis='y2'))

    # Gráfico de lineas para el número de invoices por mes en el eje y derecho
    fig.add_trace(go.Scatter(x=df_country['month_year'], y=df_country['invoiceno'], mode='lines+markers', marker=dict(size=8), line=dict(color='black'), name='# invoices', yaxis='y2'))

    # Configurar el diseño del gráfico
    fig.update_layout(title_text=f'Flujo de inventario e ingresos por mes en {country}', xaxis_title='Meses', yaxis=dict(title='Total ingresos'), yaxis2=dict(title='# Items/Invoices', overlaying='y', side='right'), hovermode='closest')

    fig.show()

Ingrese el pais en la funcion para consultar la data mensual

In [319]:
countries_list

array(['united kingdom', 'france', 'australia', 'netherlands', 'germany',
       'norway', 'eire', 'switzerland', 'spain', 'poland', 'portugal',
       'italy', 'belgium', 'lithuania', 'japan', 'iceland',
       'channel islands', 'denmark', 'cyprus', 'sweden', 'austria',
       'israel', 'finland', 'bahrain', 'greece', 'hong kong', 'singapore',
       'lebanon', 'united arab emirates', 'saudi arabia',
       'czech republic', 'canada', 'unspecified', 'brazil', 'usa',
       'european community', 'malta', 'rsa'], dtype=object)

In [322]:
plot_country_data('singapore')

In [220]:
# Trafico de usuarios por hora y numero de invoices
hour_count_invoices = df_countries.groupby(['hour'])['invoiceno'].count().reset_index()
user_count_buys = df_countries.groupby(['hour'])['customerid'].nunique().reset_index()

users_hour_invoices = pd.merge(hour_count_invoices, user_count_buys, on=['hour'])
users_hour_invoices = users_hour_invoices.rename(columns = {'invoiceno':'num_invoices','customerid':'num_customers'})
users_hour_invoices = users_hour_invoices.sort_values(by='hour', ascending = True)
users_hour_invoices

,hour,num_invoices,num_customers
0,6,14,3
1,7,165,6
2,8,2365,73
3,9,4975,144
4,10,7041,173
5,11,4841,136
6,12,6430,155
7,13,6389,163
8,14,6166,135
9,15,4286,133


In [221]:
# Crear el gráfico con Plotly
fig = go.Figure()
fig.add_trace(go.Bar(x=users_hour_invoices['hour'], y=users_hour_invoices['num_invoices'], marker=dict(color='blue'), name='total compras'))
fig.add_trace(go.Scatter(x=users_hour_invoices['hour'], y=users_hour_invoices['num_customers'], mode='lines+markers', marker=dict(size=6), line=dict(color='red'), name='# customers', yaxis='y2'))

fig.update_layout(title_text='Actividad de usuarios por hora', xaxis_title='hora', yaxis=dict(title='total compras'), yaxis2=dict(title='# customers', overlaying='y', side='right'), hovermode='closest')
fig.show()

### Descriptivo comportamiento de usuarios

In [267]:
not_greater_invoice_countries['avg_quantity'] = (not_greater_invoice_countries['quantity'] / not_greater_invoice_countries['invoices']).round(2)
not_greater_invoice_countries['avg_total_pay'] = (not_greater_invoice_countries['total_pay'] / not_greater_invoice_countries['invoices']).round(2)
not_greater_invoice_countries.head()

,country,invoices,% invoices,total_pay,quantity,customers,latitude,longitude,avg_quantity,avg_total_pay
36,saudi arabia,10,0.002,160.67,85,1,23.885942,45.079162,8.50,16.07
35,bahrain,19,0.004,959.88,368,2,26.066700,50.557700,19.37,50.52
34,czech republic,30,0.006,945.76,750,1,49.817492,15.472962,25.00,31.53
33,brazil,32,0.006,1143.60,356,1,-14.235004,-51.925280,11.12,35.74
32,lithuania,35,0.007,1661.06,652,1,55.169438,23.881275,18.63,47.46


In [277]:

trace_customers = go.Scatter(x=not_greater_invoice_countries['country'], y=not_greater_invoice_countries['customers'], mode='lines',  marker=dict(size=4), line=dict(color='black'), name='num_clientes')
trace_avg_quantity = go.Scatter(x=not_greater_invoice_countries['country'], y=not_greater_invoice_countries['avg_quantity'], mode='lines', marker=dict(size=4), line=dict(color='blue'), name='promedio_num_items')
trace_avg_total_pay = go.Scatter(x=not_greater_invoice_countries['country'], y=not_greater_invoice_countries['avg_total_pay'], mode='lines',  marker=dict(size=4), line=dict(color='red'), name='promedio_pago')


layout = go.Layout(title='Comportamiento transaccional promedio de usuario por pais',
                   xaxis=dict(title='country', tickangle=270),
                   yaxis=dict(title='nums'),
                   barmode='group')

# Crear la figura con las trazas y el layout
fig = go.Figure(data=[trace_customers, trace_avg_quantity, trace_avg_total_pay], layout=layout)

# Mostrar el gráfico
fig.show()